In [82]:
import pandas as pd
import requests
from datetime import datetime
import duckdb
import pyarrow
import os



URL = "https://ll.thespacedevs.com/2.3.0/launches/"

params = {
   
    "lsp__name": "SpaceX",   
    "window_start__gte": "2024-01-01",  
    "window_start__lte": "2025-12-31", 
    "ordering": "-window_start",
    "limit": 5,                    
    "format": "json"
}    

resp = requests.get(URL, params=params, timeout=30)
resp.raise_for_status()

data = resp.json()


In [22]:
launches = data["results"]


In [23]:
df = pd.json_normalize(launches)

In [25]:
df.to_csv("rocket.csv", sep=";", header=True, index=False)

In [34]:
con = duckdb.connect()


In [35]:
con.execute("""
    SELECT 
        *
    FROM rocket.csv
""").df()

,id,url,name,response_mode,slug,launch_designator,last_updated,net,window_end,window_start,...,pad.location.image.license.priority,pad.location.image.license.link,pad.location.image.single_use,pad.location.image.variants,pad.location.map_image,pad.location.longitude,pad.location.latitude,pad.location.timezone_name,pad.location.total_launch_count,pad.location.total_landing_count
0,161dbc75-e973-44f7-a346-6254aec99900,https://ll.thespacedevs.com/2.3.0/launches/161...,Falcon 9 Block 5 | Starlink Group 7-9,normal,falcon-9-block-5-starlink-group-7-9,2024-002,2024-03-05 18:11:38,2024-01-03 03:44:20,2024-01-03 06:02:10,2024-01-03 02:13:00,...,0,https://eur-lex.europa.eu/legal-content/EN/TXT...,True,[],https://thespacedevs-prod.nyc3.digitaloceanspa...,-120.520230,34.751330,America/Los_Angeles,855,33
1,1d374bef-0e4e-4b3f-b45b-25c14aaa2b0e,https://ll.thespacedevs.com/2.3.0/launches/1d3...,Falcon 9 Block 5 | Ovzon-3,normal,falcon-9-block-5-ovzon-3,2024-003,2024-03-05 17:33:08,2024-01-03 23:04:00,2024-01-03 23:14:00,2024-01-03 23:04:00,...,0,https://www.nasa.gov/nasa-brand-center/images-...,True,[],https://thespacedevs-prod.nyc3.digitaloceanspa...,-80.577778,28.488889,America/New_York,1081,70
2,60634a63-633d-42b3-b87e-710e7e0d4282,https://ll.thespacedevs.com/2.3.0/launches/606...,Falcon 9 Block 5 | Starlink Group 6-35,normal,falcon-9-block-5-starlink-group-6-35,2024-005,2024-03-05 18:05:37,2024-01-07 22:35:40,2024-01-08 00:35:10,2024-01-07 21:00:40,...,0,https://www.nasa.gov/nasa-brand-center/images-...,True,[],https://thespacedevs-prod.nyc3.digitaloceanspa...,-80.577778,28.488889,America/New_York,1081,70
3,429277e6-dc65-4166-a556-44edc39d71ea,https://ll.thespacedevs.com/2.3.0/launches/429...,Falcon 9 Block 5 | Starlink Group 7-10,normal,falcon-9-block-5-starlink-group-7-10,2024-011,2024-03-05 17:50:58,2024-01-14 08:59:30,2024-01-14 08:59:30,2024-01-14 08:59:30,...,0,https://eur-lex.europa.eu/legal-content/EN/TXT...,True,[],https://thespacedevs-prod.nyc3.digitaloceanspa...,-120.520230,34.751330,America/Los_Angeles,855,33
4,839c1427-45db-4b88-ae41-bb666c1dd097,https://ll.thespacedevs.com/2.3.0/launches/839...,Falcon 9 Block 5 | Starlink Group 6-37,normal,falcon-9-block-5-starlink-group-6-37,2024-012,2024-03-05 18:00:37,2024-01-15 01:52:00,2024-01-15 04:23:40,2024-01-15 00:27:00,...,0,https://www.nasa.gov/nasa-brand-center/images-...,True,[],https://thespacedevs-prod.nyc3.digitaloceanspa...,-80.577778,28.488889,America/New_York,1081,70


In [91]:
df.to_parquet("rocket.parquet")

In [61]:
pd.read_parquet("rocket.parquet")

,id,url,name,response_mode,slug,launch_designator,last_updated,net,window_end,window_start,...,pad.location.image.license.priority,pad.location.image.license.link,pad.location.image.single_use,pad.location.image.variants,pad.location.map_image,pad.location.longitude,pad.location.latitude,pad.location.timezone_name,pad.location.total_launch_count,pad.location.total_landing_count
0,161dbc75-e973-44f7-a346-6254aec99900,https://ll.thespacedevs.com/2.3.0/launches/161...,Falcon 9 Block 5 | Starlink Group 7-9,normal,falcon-9-block-5-starlink-group-7-9,2024-002,2024-03-05T18:11:38Z,2024-01-03T03:44:20Z,2024-01-03T06:02:10Z,2024-01-03T02:13:00Z,...,0,https://eur-lex.europa.eu/legal-content/EN/TXT...,True,[],https://thespacedevs-prod.nyc3.digitaloceanspa...,-120.520230,34.751330,America/Los_Angeles,855,33
1,1d374bef-0e4e-4b3f-b45b-25c14aaa2b0e,https://ll.thespacedevs.com/2.3.0/launches/1d3...,Falcon 9 Block 5 | Ovzon-3,normal,falcon-9-block-5-ovzon-3,2024-003,2024-03-05T17:33:08Z,2024-01-03T23:04:00Z,2024-01-03T23:14:00Z,2024-01-03T23:04:00Z,...,0,https://www.nasa.gov/nasa-brand-center/images-...,True,[],https://thespacedevs-prod.nyc3.digitaloceanspa...,-80.577778,28.488889,America/New_York,1081,70
2,60634a63-633d-42b3-b87e-710e7e0d4282,https://ll.thespacedevs.com/2.3.0/launches/606...,Falcon 9 Block 5 | Starlink Group 6-35,normal,falcon-9-block-5-starlink-group-6-35,2024-005,2024-03-05T18:05:37Z,2024-01-07T22:35:40Z,2024-01-08T00:35:10Z,2024-01-07T21:00:40Z,...,0,https://www.nasa.gov/nasa-brand-center/images-...,True,[],https://thespacedevs-prod.nyc3.digitaloceanspa...,-80.577778,28.488889,America/New_York,1081,70
3,429277e6-dc65-4166-a556-44edc39d71ea,https://ll.thespacedevs.com/2.3.0/launches/429...,Falcon 9 Block 5 | Starlink Group 7-10,normal,falcon-9-block-5-starlink-group-7-10,2024-011,2024-03-05T17:50:58Z,2024-01-14T08:59:30Z,2024-01-14T08:59:30Z,2024-01-14T08:59:30Z,...,0,https://eur-lex.europa.eu/legal-content/EN/TXT...,True,[],https://thespacedevs-prod.nyc3.digitaloceanspa...,-120.520230,34.751330,America/Los_Angeles,855,33
4,839c1427-45db-4b88-ae41-bb666c1dd097,https://ll.thespacedevs.com/2.3.0/launches/839...,Falcon 9 Block 5 | Starlink Group 6-37,normal,falcon-9-block-5-starlink-group-6-37,2024-012,2024-03-05T18:00:37Z,2024-01-15T01:52:00Z,2024-01-15T04:23:40Z,2024-01-15T00:27:00Z,...,0,https://www.nasa.gov/nasa-brand-center/images-...,True,[],https://thespacedevs-prod.nyc3.digitaloceanspa...,-80.577778,28.488889,America/New_York,1081,70


In [95]:
sql_time = datetime.fromtimestamp(os.path.getmtime('rocket.parquet')).strftime('%Y-%m-%d %H:%M:%S')

con.execute("""
    SELECT 
        name,
        launch_designator,
        last_updated,
        window_end,
        window_start,
        "pad.location.timezone_name",
        ? as updated_at
    FROM rocket.parquet
""", [sql_time]).df()

,name,launch_designator,last_updated,window_end,window_start,pad.location.timezone_name,updated_at
0,Falcon 9 Block 5 | Starlink Group 7-9,2024-002,2024-03-05T18:11:38Z,2024-01-03T06:02:10Z,2024-01-03T02:13:00Z,America/Los_Angeles,2026-01-19 18:19:27
1,Falcon 9 Block 5 | Ovzon-3,2024-003,2024-03-05T17:33:08Z,2024-01-03T23:14:00Z,2024-01-03T23:04:00Z,America/New_York,2026-01-19 18:19:27
2,Falcon 9 Block 5 | Starlink Group 6-35,2024-005,2024-03-05T18:05:37Z,2024-01-08T00:35:10Z,2024-01-07T21:00:40Z,America/New_York,2026-01-19 18:19:27
3,Falcon 9 Block 5 | Starlink Group 7-10,2024-011,2024-03-05T17:50:58Z,2024-01-14T08:59:30Z,2024-01-14T08:59:30Z,America/Los_Angeles,2026-01-19 18:19:27
4,Falcon 9 Block 5 | Starlink Group 6-37,2024-012,2024-03-05T18:00:37Z,2024-01-15T04:23:40Z,2024-01-15T00:27:00Z,America/New_York,2026-01-19 18:19:27
